In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from preprocessor import Preprocessor
from evaluator import ModelEvaluator
import pandas as pd

In [2]:
'''
Работа с категориальными фичами. Удаляем 'XNA' из гендера, 'Unknown' из семейного статуса, 'Maternity leave' из NAME_INCOME_TYPE (нет в тесте).
Заменяем образование и семейный статус на порядковые номера в порядке доли дефолта по этим фичам (на основе eda).
Бинарные категориальные фичи превращаем в булевы переменные, остальные в дамми.
'''

"\nРабота с категориальными фичами. Удаляем 'XNA' из гендера, 'Unknown' из семейного статуса, 'Maternity leave' из NAME_INCOME_TYPE (нет в тесте).\nЗаменяем образование и семейный статус на порядковые номера в порядке доли дефолта по этим фичам (на основе eda).\nБинарные категориальные фичи превращаем в булевы переменные, остальные в дамми.\n"

In [4]:
df_train = pd.read_csv("C:/Users/Роман/Chepenkov_dz2/application_train.csv")
df_test = pd.read_csv("C:/Users/Роман/Chepenkov_dz2/application_test.csv")

In [5]:
def hypothesis_1(df):
    df_train = df.copy()
    cat_cols = df.select_dtypes(include=['object', 'string']).columns
    num_cols = df.select_dtypes(include=['float64', 'int64']).columns
    df = df[df["CODE_GENDER"] != 'XNA']
    df = df[df["NAME_FAMILY_STATUS"] != 'Unknown']
    df = df[df['NAME_INCOME_TYPE'] != 'Maternity leave']

    education_order = ['Lower secondary','Secondary / secondary special','Incomplete higher', 'Higher education', 'Academic degree']
    df["NAME_EDUCATION_TYPE"] = df["NAME_EDUCATION_TYPE"].map(
    {v: i for i, v in enumerate(education_order)}
    )

    family_order = ['Civil marriage','Single / not married','Separated', 'Married', 'Widow']
    df["NAME_FAMILY_STATUS"] = df["NAME_FAMILY_STATUS"].map(
    {v: i for i, v in enumerate(family_order)}
    )

    for col in df[cat_cols]:
        if df[col].nunique() == 2:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])
        
    df = pd.get_dummies(df, columns=[col for col in cat_cols if df[col].nunique() > 2],dtype=int)
    
    return df

In [6]:
preprocessor = Preprocessor(custom_steps=[hypothesis_1])

In [7]:
df_train = df_train[df_train["CODE_GENDER"] != 'XNA']
df_train = df_train[df_train["NAME_FAMILY_STATUS"] != 'Unknown']
df_train = df_train[df_train['NAME_INCOME_TYPE'] != 'Maternity leave']

In [8]:
X_train_processed = preprocessor.transform(df_train.drop(columns=['TARGET', 'SK_ID_CURR']))
num_cols = X_train_processed.select_dtypes(include='number').columns
X_train_processed = X_train_processed[num_cols]

X_test_processed = preprocessor.transform(df_test.drop(columns=['SK_ID_CURR']))
X_test_processed = X_test_processed[num_cols]

In [11]:
evaluator = ModelEvaluator(LogisticRegression(max_iter=100))

roc_auc = evaluator.evaluate(X_train_processed, df_train['TARGET'])
print(f"ROC-AUC train: {roc_auc:.5f}")

evaluator.save_test_predictions(
    X_test=X_test_processed,
    ids=df_test['SK_ID_CURR'],
    filename='test_predictions_hyp_1.csv'
)
#ROC-AUC train: 0.61943

#ROC-AUC test: 0.60910
#ROC-AUC test baseline: 0.61204
#ROC-AUC test surplus: -0,00294


C:\Users\Роман\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ROC-AUC train: 0.61943
